Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/My Drive/case study-job title/'

Read the Occupation Data 

In [7]:
import pandas as pd
df_occup = pd.read_excel(path+'Occupation Data.xlsx')
df_occup['Title'].value_counts()

Audio and Video Equipment Technicians                1
Radio Frequency Identification Device Specialists    1
Pile-Driver Operators                                1
Farm Equipment Mechanics and Service Technicians     1
Industrial Ecologists                                1
                                                    ..
Curators                                             1
Infantry                                             1
Electromechanical Engineering Technologists          1
Postal Service Clerks                                1
Telecommunications Engineering Specialists           1
Name: Title, Length: 1110, dtype: int64

In [8]:
occup_title = set(list(df_occup['Title']))

Reading the Skills data

In [9]:
df_skills = pd.read_excel(path+'Skills.xlsx')
df_skills['Title'].value_counts()
df_skills.head()

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8.0,0.16,4.43,5.07,N,N,07/2014,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.88,8.0,0.23,4.43,5.32,N,N,07/2014,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.00,8.0,0.00,4.00,4.00,N,NaN,07/2014,Analyst


In [10]:

df_skills['Element Name'].value_counts()

Active Learning                      1936
Mathematics                          1936
Active Listening                     1936
Social Perceptiveness                1936
Persuasion                           1936
Management of Personnel Resources    1936
Instructing                          1936
Repairing                            1936
Quality Control Analysis             1936
Equipment Selection                  1936
Installation                         1936
Systems Evaluation                   1936
Equipment Maintenance                1936
Speaking                             1936
Operation Monitoring                 1936
Reading Comprehension                1936
Learning Strategies                  1936
Programming                          1936
Time Management                      1936
Management of Financial Resources    1936
Monitoring                           1936
Judgment and Decision Making         1936
Management of Material Resources     1936
Writing                           

Considering only those skills for respective title if importance of that skill is greater than 0

In [12]:
dict_title_skill={}
skills = list(df_skills['Element Name'])
job_title = list(df_skills['Title'])
job_title_set = list(set(df_skills['Title']))
scale_name = list(df_skills['Scale Name'])
data_value = list(df_skills['Data Value'])

for i in occup_title:
    reqd_list = []
    for j in range(len(job_title)):
        #print('{} : {}'.format(scale_name[j],data_value[j]))
        if job_title[j]==i: 
            if scale_name[j]=='Importance' and data_value[j]>0:
                #print('here')   
                reqd_list.append(skills[j])
    
    
    listToStr = ', '.join([str(elem) for elem in reqd_list])
    dict_title_skill[i]=listToStr

Creating data frame containing Title and important skills

In [13]:
df_title_skill=pd.DataFrame.from_dict(dict_title_skill,orient ='index')
#df_title_skill.to_csv(path+'/title_skill.csv')

# Renaming the column names
df_title_skill['Title']=df_title_skill.index
df_title_skill.columns=['Skills','Title']
df_title_skill


,Skills,Title
Telecommunications Engineering Specialists,"Reading Comprehension, Active Listening, Writi...",Telecommunications Engineering Specialists
"Chemistry Teachers, Postsecondary","Reading Comprehension, Active Listening, Writi...","Chemistry Teachers, Postsecondary"
Transportation Security Screeners,"Reading Comprehension, Active Listening, Writi...",Transportation Security Screeners
Range Managers,"Reading Comprehension, Active Listening, Writi...",Range Managers
Loan Counselors,"Reading Comprehension, Active Listening, Writi...",Loan Counselors
...,...,...
"Transportation Attendants, Except Flight Attendants","Reading Comprehension, Active Listening, Writi...","Transportation Attendants, Except Flight Atten..."
Command and Control Center Officers,,Command and Control Center Officers
"Musicians, Instrumental","Reading Comprehension, Active Listening, Writi...","Musicians, Instrumental"
Instructional Coordinators,"Reading Comprehension, Active Listening, Writi...",Instructional Coordinators


Merge Occupation data frame with Important Skills

In [118]:
df_orig = pd.merge(df_occup, df_title_skill, on=['Title'])
df = df_orig
df.head()

,O*NET-SOC Code,Title,Description,Skills
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,"Reading Comprehension, Active Listening, Writi..."
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, sh...","Reading Comprehension, Active Listening, Writi..."
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ...","Reading Comprehension, Active Listening, Writi..."
3,11-1031.00,Legislators,"Develop, introduce or enact laws and statutes ...",
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...","Reading Comprehension, Active Listening, Writi..."


In [15]:
df = df.drop(columns=['O*NET-SOC Code'])
df.head()

,Title,Description,Skills
0,Chief Executives,Determine and formulate policies and provide o...,"Reading Comprehension, Active Listening, Writi..."
1,Chief Sustainability Officers,"Communicate and coordinate with management, sh...","Reading Comprehension, Active Listening, Writi..."
2,General and Operations Managers,"Plan, direct, or coordinate the operations of ...","Reading Comprehension, Active Listening, Writi..."
3,Legislators,"Develop, introduce or enact laws and statutes ...",
4,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...","Reading Comprehension, Active Listening, Writi..."


# Removing the stop words and lemmatizing the words present in Description Column

Preprocessing the data

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['Description'] = df['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

Lemmatization

In [18]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['Description'] = df['Description'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [19]:
corpus = list(df['Description'])

One hot encoding of Description column

In [28]:
from keras.preprocessing.text import one_hot

# Declaring vocabulary size as 5000
voc_size = 5000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

Make all the sentences in the columns of equal length

In [29]:
from keras.preprocessing.sequence import pad_sequences 
sent_length = 70
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)


Creating a column having index of each title which we will be used as output

In [30]:
import numpy as np
data = np.arange(0, 1110)
data = pd.Series(data)
df['Title_Encoded']=data
df.head()

,Title,Description,Skills,Title_Encoded
0,Chief Executives,determin formul polici provid overal direct co...,"Reading Comprehension, Active Listening, Writi...",0
1,Chief Sustainability Officers,"commun coordin management, shareholders, custo...","Reading Comprehension, Active Listening, Writi...",1
2,General and Operations Managers,"plan, direct, coordin oper public privat secto...","Reading Comprehension, Active Listening, Writi...",2
3,Legislators,"develop, introduc enact law statut local, trib...",,3
4,Advertising and Promotions Managers,"plan, direct, coordin advertis polici program ...","Reading Comprehension, Active Listening, Writi...",4


## Will train the model on entire data present (since each Title is present only once in the data frame).

Making Embedded doc as input and Title_Encoded as dependent variable(output)

In [31]:
y = df['Title_Encoded']
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=30)

Making entire dataset as train data

In [33]:
X_train = X_final
y_train = y_final


Creating models

Creating sequential model. 



1.   Since still I am in learning phase, so tried hit and trial method for layers to be added. 
2.   Also tried with optimizer as 'adam' instead of 'rmsprop', but 'adam' resulted in less accuracy.

 

In [141]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras.models import Sequential
from keras import layers
from keras import regularizers
embedding_vector_features=70
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(layers.Dense(64, activation='relu'))
model.add(Dense(1110,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 70, 70)            350000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 100)               68400     
_________________________________________________________________
dense_23 (Dense)             (None, 64)                6464      
_________________________________________________________________
dense_24 (Dense)             (None, 1110)              72150     
Total params: 497,014
Trainable params: 497,014
Non-trainable params: 0
_________________________________________________________________
None


Converting dependent variable to categorical feature and passing number of possible classes

In [142]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
y_train_oh = to_categorical(y_train,num_classes=1110)
y_test_oh = to_categorical(y_test,num_classes=1110)

Tried with different batch sizes : 64,128,512...out of which 32 gave the best result

In [143]:
### Finally Training
model.fit(X_train,y_train_oh,validation_data=(X_test,y_test_oh),epochs=40,batch_size=32)

Epoch 1/40
35/35 [==============================] - 3s 96ms/step - loss: 7.0201 - accuracy: 0.0000e+00 - val_loss: 7.0109 - val_accuracy: 0.0082
Epoch 2/40
35/35 [==============================] - 3s 79ms/step - loss: 7.0130 - accuracy: 0.0027 - val_loss: 7.0016 - val_accuracy: 0.0218
Epoch 3/40
35/35 [==============================] - 2s 71ms/step - loss: 6.9950 - accuracy: 0.0036 - val_loss: 6.9452 - val_accuracy: 0.0054
Epoch 4/40
35/35 [==============================] - 2s 70ms/step - loss: 6.8787 - accuracy: 0.0045 - val_loss: 6.7060 - val_accuracy: 0.0000e+00
Epoch 5/40
35/35 [==============================] - 2s 71ms/step - loss: 6.6614 - accuracy: 0.0018 - val_loss: 6.4502 - val_accuracy: 0.0027
Epoch 6/40
35/35 [==============================] - 3s 72ms/step - loss: 6.4032 - accuracy: 0.0018 - val_loss: 6.1214 - val_accuracy: 0.0027
Epoch 7/40
35/35 [==============================] - 3s 72ms/step - loss: 6.1009 - accuracy: 0.0018 - val_loss: 5.7687 - val_accuracy: 0.0136
Epoch

Create function to predict the job title and skills based on description

In [107]:
def pred_result(data,model):

    # Perform cleaning of input data before prediction
    # Remove Stop Word
    
    data = data.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Lemmatization
    data = data.apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    data_list = list(data)
    voc_size = 5000
    one_hot_repr = [one_hot(words,voc_size) for words in data_list]
    sent_length = 70
    embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)    
    X_pred=np.array(embedded_docs)
    model.fit(X_train,y_train_oh,epochs=40,batch_size=32, verbose=0)
    results = model.predict_classes(X_pred)
    return results

Result contains the index of all the job title from which we can get job title as well as skills

In [108]:
result = pred_result(df['Description'],model)

Create predicted data frame

In [ ]:
pred_title = []
pred_skill = []
for i in range(len(result)):
    pred_title.append(df_orig.iloc[i,1])
    pred_skill.append(df_orig.iloc[i,3])


In [129]:
predicted_Title = pd.Series(pred_title)
predicted_Skills = pd.Series(pred_skill)

data = {'Description':  list(df_orig['Description']),
        'Predicted_Title': predicted_Title,
        'Predicted_Skills': predicted_Skills 
        }

df_predicted = pd.DataFrame (data, columns = ['Description','Predicted_Title','Predicted_Skills'])



df_predicted.head()

,Description,Predicted_Title,Predicted_Skills
0,Determine and formulate policies and provide o...,Chief Executives,"Reading Comprehension, Active Listening, Writi..."
1,"Communicate and coordinate with management, sh...",Chief Sustainability Officers,"Reading Comprehension, Active Listening, Writi..."
2,"Plan, direct, or coordinate the operations of ...",General and Operations Managers,"Reading Comprehension, Active Listening, Writi..."
3,"Develop, introduce or enact laws and statutes ...",Legislators,
4,"Plan, direct, or coordinate advertising polici...",Advertising and Promotions Managers,"Reading Comprehension, Active Listening, Writi..."


In [131]:
df_predicted.to_csv(path+'/Prediction_deliver.csv')